We received all of the transient booking records from the Boulder County Sheriff's office from 2000-2016. They were sent with the following notes:

* The data provided may not have all the fields complete due to old system conversion to new.
* The request asked to match the data elements which are available from the “Public” tool, which includes associated charges, and to provide the data in an Excel format. To meet both these expectations required resolving a “many-to-one” problem (many charges to one offender) which results in there potentially being multiple Excel “Row” records for each booking number

Now we will explore the shape of this data, and join it with the raw jail listings.

Several goals of this process are to:
* Flatten the booking records so that a single person getting booked is a single row
* Add a column for whether or not that person is transient

In [2]:
import pandas as pd

df_sheriff = pd.read_excel('../data/transient-bookings-records.xlsx')

In [20]:
pd.set_option("display.max_columns", 100)

In [140]:
df_bookings = pd.read_csv('../data/all-bookings-with-antihomeless-charges.csv')

In [141]:
print('''The sheriff dataset has {} rows.'''.format(df_sheriff.shape[0]))

df_sheriff.head(3)

The sheriff dataset has 54469 rows.


,Booking Number,Name,Address,City,State,ZIP Code,Race,Sex,DOB,Booking Date,Booking Time,Facility,Location,Arresting Agency,Charge,Title,Charge Level
0,58,"SAUL,KENNETH BRETT",NaN,NaN,CO,NaN,W,M,1975-10-19,2000-01-02,245,NaN,NaN,LOPD,18 3 208,RECKLESS ENDAGERMENT,M3
1,58,"SAUL,KENNETH BRETT",NaN,NaN,CO,NaN,W,M,1975-10-19,2000-01-02,245,NaN,NaN,LOPD,16 11 502.1,FTC: FTC VIOLATION O,X
2,85,"MORIN,ARISTEO CULONGO",NaN,NONE,CO,NaN,W,M,1957-09-10,2000-01-03,46,NaN,NaN,LFPD,18 5 113,CRIM IMPERSONATION,F6


In [142]:
# These charge levels are explained here: http://www.mtnlegal.com/2012/10/07/guide-to-criminal-penalties-in-colorado/

df_sheriff['Charge Level'].value_counts()

MO    17985
X      8256
M      4648
M2     4486
M3     3752
M1     2950
F4     2839
F5     2134
F6     1730
F      1581
P1     1111
P2     1087
F3     1062
T       541
F2      123
F1       37
I         3
Name: Charge Level, dtype: int64

In [143]:
df_bookings.head(3)

,Name,Booking No,Booked,Location,DOB,Race,Sex,Case No,Arresting Agency,Charge,Arrest Date,charge_text,legal_code_parts,legal_code_no_1,legal_code_no_2,legal_code_no_3,legal_code_no_4,camping,fta,ftc,booking_time,boulder,urination,vehicle_as_residence,public_obstruct,public_trespass,begging,antihomeless,smoking
0,"ARELLANO-ORDAZ,SIMON",1106625,2011-08-09 22:20:00,BJ INW,1988-04-10,W,M,110010043,BOULDER PD,18-18-405(2)(A)(I). SALE/MFG/DIST/CONT S,2011-08-09,SALE/MFG/DIST/CONT S,"(18, 18, 405, 2)",18.0,18.0,405.0,2.0,False,False,False,2011-08-09 22:20:00,False,False,False,False,False,False,False,False
1,"ARELLANO-ORDAZ,SIMON",1106625,2011-08-09 22:20:00,BJ INW,1988-04-10,W,M,110010043,BOULDER PD,18-6-401(7)(B)(I) CHILD ABUSE,2011-08-09,CHILD ABUSE,"(18, 6, 401, 7)",18.0,6.0,401.0,7.0,False,False,False,2011-08-09 22:20:00,False,False,False,False,False,False,False,False
2,"ARELLANO-ORDAZ,SIMON",1106625,2011-08-09 22:20:00,BJ INW,1988-04-10,W,M,110010043,BOULDER PD,42-2-101(1) DRIVING WITHOUT A VA,2011-08-09,DRIVING WITHOUT A VA,"(42, 2, 101, 1)",42.0,2.0,101.0,1.0,False,False,False,2011-08-09 22:20:00,False,False,False,False,False,False,False,False


In [144]:
df_sheriff_by_person = df_sheriff.groupby('Booking Number').first()
df_bookings_by_person = df_bookings.groupby('Booking No').first()

In [145]:
df_bookings_by_person['any_antihomeless'] = df_bookings.groupby('Booking No').antihomeless.any()

In [146]:
antihomeless_columns =  ["smoking", "camping", "urination", "vehicle_as_residence", "public_obstruct", "public_trespass", "begging"]

In [147]:
df_bookings_by_person_antihomeless = df_bookings.groupby('Booking No')[antihomeless_columns].any()

In [148]:
df_bookings_by_person = pd.concat([df_bookings_by_person, df_bookings_by_person_antihomeless], axis=1)

In [149]:
df_bookings_by_person.drop([
        'Charge', 'charge_text', 'legal_code_parts', 'legal_code_no_1', 'legal_code_no_2', 'legal_code_no_3',
        'legal_code_no_4'
    ], axis=1, inplace=True)

In [150]:
print('''Unique booking ids in

transient dataset: {}
overall dataset: {}'''.format(
        df_sheriff_by_person.shape[0],
        df_bookings_by_person.shape[0]
))

Unique booking ids in

transient dataset: 21962
overall dataset: 167633


In [151]:
df_bookings_sheriff = pd.merge(df_bookings_by_person, df_sheriff_by_person, how='left', left_index=True, right_index=True, suffixes=('', '_s'))

In [152]:
print('''The merged column has {} rows.

{} of the rows are from the transient table
'''.format(
        df_bookings_sheriff.shape[0], 
        df_bookings_sheriff['Name_s'].count()
    )
)

The merged column has 167633 rows.

21895 of the rows are from the transient table



Not too bad. We lost 67 records between the two datasets.

In [153]:
df_bookings_sheriff.head()

,Name,Booked,Location,DOB,Race,Sex,Case No,Arresting Agency,Arrest Date,camping,fta,ftc,booking_time,boulder,urination,vehicle_as_residence,public_obstruct,public_trespass,begging,antihomeless,smoking,any_antihomeless,smoking,camping,urination,vehicle_as_residence,public_obstruct,public_trespass,begging,Name_s,Address,City,State,ZIP Code,Race_s,Sex_s,DOB_s,Booking Date,Booking Time,Facility,Location_s,Arresting Agency_s,Charge,Title,Charge Level
Booking No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,"HOOD,AARON JAY",2000-01-01 02:12:00,,1975-10-08,W,M,991126052,UNIVERSITY OF COLORADO,1999-12-31,False,True,False,2000-01-01 02:12:00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"LAWYER,KENNETH A",2000-01-01 04:01:00,,1958-09-29,W,M,991001313,JAIL MITTS ONLY,2000-01-16,False,False,False,2000-01-01 04:01:00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"AGUILAR TORRES,MIGUEL",2000-01-01 03:11:00,,1972-11-14,W,M,991126053,UNIVERSITY OF COLORADO,1999-12-31,False,False,False,2000-01-01 03:11:00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"JUDD,OLIVER RANDALL",2000-01-01 01:25:00,,1953-06-07,W,M,991031932,BOULDER PD,1999-12-31,False,False,False,2000-01-01 01:25:00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"WINTER,STEPHEN SHANE",2000-01-01 03:49:00,,1961-09-09,W,M,991102428,LONGMONT PD,1999-12-31,False,False,False,2000-01-01 03:49:00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
df_bookings_sheriff = df_bookings_sheriff.loc[:,~df_bookings_sheriff.columns.duplicated()]

In [155]:
df_bookings_sheriff['transient'] = ~df_bookings_sheriff['Name_s'].isnull()

In [156]:
droppable_columns = [x for x in df_bookings_sheriff.columns if x.endswith('_s')]

droppable_columns

['Name_s', 'Race_s', 'Sex_s', 'DOB_s', 'Location_s', 'Arresting Agency_s']

In [157]:
df_bookings_sheriff.drop(droppable_columns + ['Charge', 'Title', 'Charge Level'], axis=1, inplace=True)

In [158]:
df_bookings_sheriff.to_csv('../data/bookings-with-transient-status.csv', index=False)